In [101]:
import numpy as np
import pandas as pd
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
import matplotlib.pyplot as plt
register_matplotlib_converters()
plt.style.use('seaborn')

In [102]:
class DataLoad:
    
    Skip_Rows_City_Time = 0
    Ski_Rows_State = 0
    BR_Cases_By_State = None
    BR_Cases_By_City = None
    BR_Cases_Total = None
    states = pd.read_csv("BR_States.csv",index_col=0)
    cities = pd.read_csv("BR_Cities_By_State.csv",encoding='utf-8',index_col=0).fillna(False)
    
    def __init__(self):
        #Imports the information from the .csv file
        df = pd.read_csv("https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-cities-time.csv",
                         encoding="utf-8", parse_dates=['date'])
        df["city"] = df["city"].str.split("/").str.get(0)+"-"+df["city"].str.split("/").str.get(1)
       
        df2 = pd.read_csv("https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv",
                         encoding="utf-8", parse_dates=['date'])
        df2["city"] = df2["city"].str.split("/").str.get(0)+"-"+df2["city"].str.split("/").str.get(1)
       
        #Saves the number of rows that are in memory
        self.Skip_Rows_City_Time = len(df.index)
        self.Ski_Rows_State = len(df2.index)
        
        #Makes hard copies from the temporary DataFrame
        self.BR_Cases_Total = df[df["state"] == "TOTAL"].copy()
        self.BR_Cases_By_City = df[df["state"] != "TOTAL"].copy()
        self.BR_Cases_By_State = df2[df2["state"] != "TOTAL"].copy()
        
        #Reindex the DataFrames
        self.BR_Cases_By_City.index = range(len(self.BR_Cases_By_City))
        self.BR_Cases_Total.index = range(len(self.BR_Cases_Total))
        self.BR_Cases_By_State.index = range(len(self.BR_Cases_By_State.index))
        
        del df
        del df2
        
    
    def __update__(self):
        #Import the informatiof from the .csv file, skiping
        #the rows that are already in memory
        df = pd.read_csv("https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-cities-time.csv",
                         encoding="utf8",skiprows=range(1,self.Skip_Rows_City_Time), parse_dates=['date'])
        df["city"] = df["city"].str.split("/").str.get(0)+"-"+df["city"].str.split("/").str.get(1)
        df2 = pd.read_csv("https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv",
                         encoding="utf-8", skiprows=range(1,self.Ski_Rows_State), parse_dates=['date'])
        df2["city"] = df2["city"].str.split("/").str.get(0)+"-"+df2["city"].str.split("/").str.get(1)
        #Updates the number of rows
        self.Skip_Rows_City_Time += len(df.index)
        self.Ski_Rows_State += len(df2.index)
        
        #Concatenates the new data to the dataframe
        self.BR_Cases_By_City = pd.concat([self.BR_Cases_Total,df[df["state"]!="TOTAL"]],ignore_index=True)
        self.BR_Cases_Total = pd.concat([self.BR_Cases_Total,df[df["state"]=="TOTAL"]],ignore_index=True)
        self.BR_Cases_By_State = pd.concat([self.BR_Cases_By_State,df2[df2["state"]!= "TOTAL"]],ignore_index=True)
        
        del df
        del df2
    

In [107]:
class DataVisualizing:
    
    BR_Cases_By_State = None
    BR_Cases_By_City = None
    BR_Cases_Total = None
    states = None
    cities = None
    def __init__(self,data):
        self.BR_Cases_By_State = data.BR_Cases_By_State
        self.BR_Cases_By_City = data.BR_Cases_By_City
        self.BR_Cases_Total = data.BR_Cases_Total
        self.states = data.states
        self.cities = data.cities
                
    
    def TemporalSeriesCity(self,city = None,hash_value=""):
        _temp = self.BR_Cases_By_City[self.BR_Cases_By_City["city"].values == city]

        Figure, Axes = plt.subplots(figsize=(8,8))

        Axes.plot_date("date","totalCases",data = _temp,linestyle="solid",label="Total Cases",color='red')
        Axes.plot_date("date","deaths",data=_temp,linestyle="solid", label="Total Deaths",color='purple')

        Axes.set_title("Temporal Series - "+city,fontsize=20)
        Axes.set_ylabel("Number",labelpad=10,fontsize=14)
        Axes.set_xlabel("Time (Days)",labelpad=10,fontsize=14)
        Axes.set_xticklabels(_temp["date"].dt.strftime("%d-%m-%Y"))

        Figure.legend(loc="upper left",bbox_to_anchor=(0.1,0.95),fontsize=12)
        Figure.autofmt_xdate()
        Figure.tight_layout()

        path = "__temp/__fixed/__tsc/tsc_"+hash_value+".png"
        Figure.savefig(u"__temp/__fixed/__tsc/tsc_"+hash_value+".png")
        del _temp
        Axes.cla()
        Figure.clear()
        plt.close()
            
    def TemporalSeriesState(self,state,hash_value=""):
        
        _temp = self.BR_Cases_By_State[self.BR_Cases_By_State["state"].values == state]
        
        Figure, Axes = plt.subplots(figsize=(8,8))

        Axes.plot_date("date","totalCases",data = _temp,linestyle="solid",label="Total Cases",color='red')
        Axes.plot_date("date","deaths",data=_temp,linestyle="solid", label="Total Deaths",color='purple')

        Axes.set_title("Temporal Series - "+state,fontsize=20)
        Axes.set_ylabel("Number",labelpad=10,fontsize=14)
        Axes.set_xlabel("Time (Days)",labelpad=10,fontsize=14)
        Axes.set_xticklabels(_temp["date"].dt.strftime("%d-%m-%Y"))

        Figure.legend(loc="upper left",bbox_to_anchor=(0.1,0.95),fontsize=12)
        Figure.autofmt_xdate()
        Figure.tight_layout()

        Figure.savefig("__temp/__fixed/__tss/tss_"+hash_value+".png")

        del _temp
        Axes.cla()
        Figure.clear()
        plt.close()
            
    
    def totalInfectedBarStates(self,hash_value=""):
        _temp = self.BR_Cases_By_State[self.BR_Cases_By_State["date"] == self.BR_Cases_By_State.date.unique()[-1]]
        l_infected = _temp.totalCases
        l_index = _temp.state

        df = pd.DataFrame({"state":l_index,"values":l_infected})
        df.sort_values("values",inplace=True)
        
        Figure, Axes = plt.subplots(figsize=(8,8))

        bar = sns.barplot("state","values",data=df,label="Total Cases",palette='inferno_r')
        
        for i in bar.patches:
            bar.annotate(str(int(i.get_height())),(i.get_x(),i.get_height()),fontsize=10,ha='left')

        Axes.set_title("Number of Infecteds by State",fontsize=20)
        Axes.set_ylabel("Number",labelpad=10,fontsize=14)
        Axes.set_xlabel("State",labelpad=10,fontsize=14)

        Figure.tight_layout()
        
        Figure.savefig("__temp/__fixed/tibs_"+hash_value+".png")
        
        del l_infected
        del l_index
        Figure.clear()
        plt.close()
        Axes.cla()
        del bar
        del _temp
        del df
        
        
    def totalInfectedBarCity(self,state,hash_value=""):
        _temp = self.BR_Cases_By_City[self.BR_Cases_By_City["date"] == self.BR_Cases_By_State.date.unique()[-1]]
        _temp = _temp[_temp["state"] == state]
        _temp = _temp[_temp["city"] != "INDEFINIDA/"+state]
        _temp.sort_values("totalCases",ascending=False,inplace=True)
        _temp = _temp.loc[_temp.index[:10]]

        l_infected = _temp.totalCases
        l_index = _temp.city
            
        df = pd.DataFrame({"city":l_index,"values":l_infected})
        df.sort_values("values",inplace=True)
        
        Figure, Axes = plt.subplots(figsize=(8,8))

        bar = sns.barplot("city","values",data=df,label="Total Cases",palette='inferno_r')
        
        for i in bar.patches:
            bar.annotate(str(int(i.get_height())),(i.get_x(),i.get_height()),fontsize=10,ha='left')

        Axes.set_title("Most Infected Cities of "+state,fontsize=20)
        Axes.set_ylabel("Number",labelpad=10,fontsize=14)
        Axes.set_xlabel("City",labelpad=10,fontsize=14)
        Axes.tick_params('x',rotation=90)
        
        Figure.tight_layout()
        
        Figure.savefig("__temp/__fixed/tibc_"+hash_value+".png")
        
        del l_infected
        del l_index
        Figure.clear()
        plt.close()
        Axes.cla()
        del bar
        del _temp
        del df

    def totalDeathsBarStates(self,hash_value=""):
        _temp = self.BR_Cases_By_State[self.BR_Cases_By_State["date"] == self.BR_Cases_By_State.date.unique()[-1]]
        l_deaths = _temp.deaths
        l_index = _temp.state

        df = pd.DataFrame({"state":l_index,"deaths":l_deaths})
        df.sort_values("deaths",inplace=True)
        
        Figure, Axes = plt.subplots(figsize=(8,8))

        bar = sns.barplot("state","deaths",data=df,label="Total Deaths",palette='inferno_r')
        
        for i in bar.patches:
            bar.annotate(str(int(i.get_height())),(i.get_x(),i.get_height()),fontsize=10,ha='left')

        Axes.set_title("Number of Deaths by State",fontsize=20)
        Axes.set_ylabel("Number",labelpad=10,fontsize=14)
        Axes.set_xlabel("State",labelpad=10,fontsize=14)

        Figure.tight_layout()
        
        Figure.savefig("__temp/__fixed/tdbs_"+hash_value+".png")

        del l_deaths
        del l_index
        Figure.clear()
        plt.close()
        Axes.cla()
        del bar
        del _temp
        del df
    
    def totalDeathsBarCity(self,state,hash_value=""):
        _temp = self.BR_Cases_By_City[self.BR_Cases_By_City["date"] == self.BR_Cases_By_State.date.unique()[-1]]
        _temp = _temp[_temp["state"] == state]
        _temp = _temp[_temp["city"] != "INDEFINIDA/"+state]
        _temp.sort_values("deaths",ascending=False,inplace=True)
        _temp = _temp.loc[_temp.index[:10]]

        l_deaths = _temp.deaths
        l_index = _temp.city
            
        df = pd.DataFrame({"city":l_index,"deaths":l_deaths})
        df.sort_values("deaths",inplace=True)
        
        Figure, Axes = plt.subplots(figsize=(8,8))

        bar = sns.barplot("city","deaths",data=df,label="Total Cases",palette='inferno_r')
        
        for i in bar.patches:
            bar.annotate(str(int(i.get_height())),(i.get_x(),i.get_height()),fontsize=10,ha='left')

        Axes.set_title("Most Death Cases in Cities of "+state,fontsize=20)
        Axes.set_ylabel("Number",labelpad=10,fontsize=14)
        Axes.set_xlabel("City",labelpad=10,fontsize=14)
        Axes.tick_params(rotation=90)
        
        Figure.tight_layout()
        
        Figure.savefig("__temp/__fixed/tdbc_"+hash_value+".png")
        
        del l_deaths
        del l_index
        Figure.clear()
        plt.close()
        Axes.cla()
        del bar
        del _temp
        del df

    def ComparisonInfectedBarState(self,state1,state2,hash_value=""):
        
        _temp = self.BR_Cases_By_State[self.BR_Cases_By_State["state"] == state1].copy()
        _temp2 = self.BR_Cases_By_State[self.BR_Cases_By_State["state"] == state2].copy()
        
        lenght  = len(_temp.index)
        lenght2 = len(_temp2.index)
        
        _temp["date"] = range(lenght)
        _temp2["date"] = range(lenght2)

        _temp.index = range(lenght)
        _temp2.index = range(lenght2)
        
        comp = lenght if lenght<=lenght2 else lenght2
        
        Figure, Axes = plt.subplots(figsize=(8,8))

        if comp > 0:
            sns.barplot("date","totalCases",'state',data=pd.concat([_temp[:comp],_temp2[:comp]]),palette='inferno')
            Axes.plot("date","totalCases",data=_temp.loc[:comp-1],color='blue',label="_")
            Axes.plot("date","totalCases",data=_temp2.loc[:comp-1],color='red',label="_")
        elif lenght >0:
            sns.barplot("date","totalCases",'state',data=_temp,palette='inferno')
            Axes.plot("date","totalCases",data=_temp.loc[:comp-1],color='blue',label="_")
        elif lenght2 >0:
            sns.barplot("date","totalCases",'state',data=_temp2,palette='inferno')
            Axes.plot("date","totalCases",data=_temp2.loc[:comp-1],color='red',label="_")
        
        
        Axes.legend(fontsize=12).set_title("States",prop={"size":14})
        Axes.set_title(state1+" x "+state2,fontsize=20)
        Axes.set_ylabel("Number",labelpad=10,fontsize=14)
        Axes.set_xlabel("Days Since First Infected",labelpad=10,fontsize=14)

        Figure.tight_layout()
        
        Figure.savefig("__temp/__fixed/__cibs/cibs_"+hash_value+".png")
        Figure.clear()
        plt.close()
        Axes.cla()
        del lenght
        del lenght2
        del _temp
        del _temp2
        del comp
        
    def ComparisonInfectedBarCity(self,city1,city2,hash_value=""):
        
        
        
        
        _temp = self.BR_Cases_By_City[self.BR_Cases_By_City["city"].values == city1].copy()
        _temp2 = self.BR_Cases_By_City[self.BR_Cases_By_City["city"].values == city2].copy()
        
        lenght  = len(_temp.index)
        lenght2 = len(_temp2.index)

        _temp["date"] = range(lenght)
        _temp2["date"] = range(lenght2)

        _temp.index = range(lenght)
        _temp2.index = range(lenght2)
        
        comp = lenght if lenght<=lenght2 else lenght2
        
        Figure, Axes = plt.subplots(figsize=(8,8))
        
        if comp>0:
                sns.barplot("date","totalCases",'city',data=pd.concat([_temp[:comp],_temp2[:comp]]),palette='inferno')
                Axes.plot("date","totalCases",data=_temp.loc[:comp-1],color='blue',label="_")
                Axes.plot("date","totalCases",data=_temp2.loc[:comp-1],color='red',label="_")
        elif lenght>0:
                sns.barplot("date","totalCases",'city',data=_temp,palette='inferno')
                Axes.plot("date","totalCases",data=_temp.loc[:comp-1],color='blue',label="_")
        elif lenght2>0:
                sns.barplot("date","totalCases",'city',data=_temp2,palette='inferno')
                Axes.plot("date","totalCases",data=_temp2.loc[:comp-1],color='red',label="_")

        

        Axes.legend(fontsize=12).set_title("Cities",prop={"size":14})
        Axes.set_title(city1+" x "+city2,fontsize=20)
        Axes.set_ylabel("Number",labelpad=10,fontsize=14)
        Axes.set_xlabel("Days Since First Infected",labelpad=10,fontsize=14)

        Figure.tight_layout()
        
        Figure.savefig("__temp/__fixed/__cibc/cibc_"+hash_value+".png")
        Figure.clear()
        plt.close()
        Axes.cla()
        del lenght
        del lenght2
        del _temp
        del _temp2
        del comp

    def ComparisonDeathsBarState(self,state1,state2,hash_value=""):
        
        _temp = self.BR_Cases_By_State[self.BR_Cases_By_State["state"] == state1].copy()
        _temp = _temp[_temp["deaths"]>0]
        _temp2 = self.BR_Cases_By_State[self.BR_Cases_By_State["state"] == state2].copy()
        _temp2 = _temp2[_temp2["deaths"]>0]
        
        lenght  = len(_temp.index)
        lenght2 = len(_temp2.index)

        _temp["date"] = range(lenght)
        _temp2["date"] = range(lenght2)

        _temp.index = range(lenght)
        _temp2.index = range(lenght2)
        
        comp = lenght if lenght<=lenght2 else lenght2
        
        Figure, Axes = plt.subplots(figsize=(8,8))

        if comp>0:
            sns.barplot("date","deaths",'state',data=pd.concat([_temp[:comp],_temp2[:comp]]),palette='inferno')
            Axes.plot("date","deaths",data=_temp.loc[:comp-1],color='blue',label="_")
            Axes.plot("date","deaths",data=_temp2.loc[:comp-1],color='red',label="_")
        elif lenght>0:
            sns.barplot("date","deaths",'state',data=_temp,palette='inferno')
            Axes.plot("date","deaths",data=_temp.loc[:comp-1],color='blue',label="_")
        elif lenght2>0:
            sns.barplot("date","deaths",'state',data=_temp2,palette='inferno')
            Axes.plot("date","deaths",data=_temp2.loc[:comp-1],color='red',label="_")
        
        Axes.legend(fontsize=12).set_title("States",prop={"size":14})
        Axes.set_title(state1+" x "+state2,fontsize=20)
        Axes.set_ylabel("Number",labelpad=10,fontsize=14)
        Axes.set_xlabel("Days Since First Death",labelpad=10,fontsize=14)

        Figure.tight_layout()
        
        Figure.savefig("__temp/__fixed/__cdbs/cdbs_"+hash_value+".png")
        Figure.clear()
        plt.close()
        Axes.cla()
        del lenght
        del lenght2
        del _temp
        del _temp2
        del comp
        
    def ComparisonDeathsBarCity(self,city1,city2,hash_value=""):
        
        
        
        
        _temp = self.BR_Cases_By_City[self.BR_Cases_By_City["city"].values == city1].copy()
        _temp = _temp[_temp["deaths"]>0]
        
        _temp2 = self.BR_Cases_By_City[self.BR_Cases_By_City["city"].values == city2].copy()
        _temp2 = _temp2[_temp2["deaths"]>0]
        
        lenght  = len(_temp.index)
        lenght2 = len(_temp2.index)

        _temp["date"] = range(lenght)
        _temp2["date"] = range(lenght2)

        _temp.index = range(lenght)
        _temp2.index = range(lenght2)
        
        comp = lenght if lenght<=lenght2 else lenght2
        
        Figure, Axes = plt.subplots(figsize=(8,8))

        if comp > 0:
            sns.barplot("date","deaths",'city',data=pd.concat([_temp[:comp],_temp2[:comp]]),palette='inferno')
            Axes.plot("date","deaths",data=_temp.loc[:comp-1],color='blue',label="_")
            Axes.plot("date","deaths",data=_temp2.loc[:comp-1],color='red',label="_")
        elif lenght>0:
            sns.barplot("date","deaths",'city',data=_temp,palette='inferno')
            Axes.plot("date","deaths",data=_temp.loc[:comp-1],color='blue',label="_")
        elif lenght2>0:
            sns.barplot("date","deaths",'city',data=_temp2,palette='inferno')
            Axes.plot("date","deaths",data=_temp2.loc[:comp-1],color='red',label="_")

        

        Axes.legend(fontsize=12).set_title("Cities",prop={"size":14})
        Axes.set_title(city1+" x "+city2,fontsize=20)
        Axes.set_ylabel("Deaths",labelpad=10,fontsize=14)
        Axes.set_xlabel("Days Since First Death",labelpad=10,fontsize=14)

        Figure.tight_layout()
        
        Figure.savefig("__temp/__fixed/__cdbc/cdbc_"+hash_value+".png")
        
        Figure.clear()
        plt.close()
        Axes.cla()
        del lenght
        del lenght2
        del _temp
        del _temp2
        del comp
        
    def HeatmapInfectedByState(self,states_list,hash_value=""):
        df = pd.DataFrame()

        comp = len(self.BR_Cases_By_State[self.BR_Cases_By_State["state"] == states_list[0]])

        for i in states_list[1:]:
            lenght = len(self.BR_Cases_By_State[self.BR_Cases_By_State["state"] == i])
            comp = lenght if lenght < comp else comp

        for i in states_list:
            _temp = self.BR_Cases_By_State[self.BR_Cases_By_State["state"] == i].copy()
            _temp["date"] = range(len(_temp.date))
            df = df.append(_temp.loc[_temp.index[:comp]])

        pivot = df.pivot('state','date','totalCases')

        Figure, Axes = plt.subplots(figsize=(8,8))
        sns.heatmap(pivot,cmap="inferno_r",fmt="",ax=Axes,annot=True,annot_kws={'rotation':90})
        Axes.set_xlabel("Days Since First Infected",fontsize=14)
        Axes.set_ylabel("_")
        Axes.tick_params(rotation=0)

        
        Figure.tight_layout()
        Figure.savefig("__temp/__custom/hibs_"+hash_value+".png")
        
        del df
        del lenght
        del comp
        del _temp
        Axes.cla()
        Figure.clear()
        plt.close()
        del pivot
    
    def HeatmapInfectedByCity(self,cities_list,hash_value=""):
        df = pd.DataFrame()
        comp = len(self.BR_Cases_By_City[self.BR_Cases_By_City["city"].values == cities_list[0].decode('utf-8')])

        for i in cities_list[1:]:
            lenght = len(self.BR_Cases_By_City[self.BR_Cases_By_City["city"].values == i.decode('utf-8')])
            comp = lenght if lenght < comp else comp

        for i in cities_list:
            _temp = self.BR_Cases_By_City[self.BR_Cases_By_City["city"].values == i.decode('utf-8')].copy()
            _temp["date"] = range(len(_temp.date))
            df = df.append(_temp.loc[_temp.index[:comp]])

        pivot = df.pivot('city','date','totalCases')

        Figure, Axes = plt.subplots(figsize=(8,8))
        sns.heatmap(pivot,cmap="inferno_r",fmt="",ax=Axes,annot=True,annot_kws={'rotation':90})
        Axes.set_xlabel("Days Since First Infected",fontsize=14)
        Axes.set_ylabel("_")
        Axes.tick_params(rotation=0)

        
        Figure.tight_layout()
        Figure.savefig("__temp/__custom/hibc_"+hash_value+".png")
        
        del df
        del lenght
        del comp
        del _temp
        Axes.cla()
        Figure.clear()
        plt.close()
        del pivot
        
    def HeatmapDeathsByState(self,states_list,hash_value=""):
        df = pd.DataFrame()
        
        bol_expr1 = self.BR_Cases_By_State["state"] == states_list[0]
        bol_expr2 = self.BR_Cases_By_State["deaths"]>0
        bol = bol_expr1 & bol_expr2
        
        comp = len(self.BR_Cases_By_State[bol])

        for i in states_list[1:]:
            bol_expr1 = self.BR_Cases_By_State["state"] == i
            bol = bol_expr1 & bol_expr2
        
            lenght = len(self.BR_Cases_By_State[bol])
            comp = lenght if lenght < comp else comp

        for i in states_list:
            bol_expr1 = self.BR_Cases_By_State["state"] == i
            bol = bol_expr1 & bol_expr2
            
            _temp = self.BR_Cases_By_State[bol].copy()
            _temp["date"] = range(len(_temp.date))
            df = df.append(_temp.loc[_temp.index[:comp]])

        pivot = df.pivot('state','date','deaths')

        Figure, Axes = plt.subplots(figsize=(8,8))
        sns.heatmap(pivot,cmap="inferno_r",fmt="",ax=Axes,annot=True,annot_kws={'rotation':90})
        Axes.set_xlabel("Days Since First Death Case",fontsize=14)
        Axes.set_ylabel("_")
        Axes.tick_params(rotation=0)

        
        Figure.tight_layout()
        Figure.savefig("__temp/__custom/hdbs_"+hash_value+".png")
        
        del df
        del lenght
        del comp
        del _temp
        Axes.cla()
        Figure.clear()
        plt.close()
        del pivot
    
    def HeatmapDeathsByCity(self,cities_list,hash_value=""):
        df = pd.DataFrame()
        bol_expr1 = self.BR_Cases_By_City["city"] == cities_list[0].decode('utf-8')
        bol_expr2 = self.BR_Cases_By_City["deaths"]>0
        bol = bol_expr1 & bol_expr2
        comp = len(self.BR_Cases_By_City[bol])

        for i in cities_list[1:]:
            
            bol_expr1 = self.BR_Cases_By_City["city"].values == i.decode('utf-8')
            bol = bol_expr1 & bol_expr2
            lenght = len(self.BR_Cases_By_City[bol])
            comp = lenght if lenght < comp else comp

        for i in cities_list:
            
            bol_expr1 = self.BR_Cases_By_City["city"].values == i.decode('utf-8')
            bol = bol_expr1 & bol_expr2
            _temp = self.BR_Cases_By_City[bol].copy()
            _temp["date"] = range(len(_temp.date))
            df = df.append(_temp.loc[_temp.index[:comp]])

        pivot = df.pivot('city','date','deaths')

        Figure, Axes = plt.subplots(figsize=(8,8))
        sns.heatmap(pivot,cmap="inferno_r",fmt="",ax=Axes,annot=True,annot_kws={'rotation':90})
        Axes.set_xlabel("Days Since First Death Case",fontsize=14)
        Axes.set_ylabel("_")
        Axes.tick_params(rotation=0)
        
        Figure.tight_layout()
        Figure.savefig("__temp/__custom/hdbc_"+hash_value+".png")
        
        del df
        del lenght
        del comp
        del _temp
        Axes.cla()
        Figure.clear()
        plt.close()
        del pivot
        
    def PieCasesByState(self):
        date = self.BR_Cases_By_State.date.unique()[-1]
        _temp = self.BR_Cases_By_State[data.BR_Cases_By_State['date'] == date].sort_values("totalCases",ascending=False)

        _main = _temp[:5]
        _others = pd.DataFrame([[None,None,"Others",None,None,None,None,sum(_temp[5:]["totalCases"]),None,None]],columns=_main.columns)
        _temp = pd.concat([_main,_others],ignore_index=True)

        colors = ['#FF214B','#FF5745','#FD6865','#FD8978','#FDA978','#DCDCDC']
        
        Figure, Axes = plt.subplots(figsize=(8,8))
        
        Axes.pie(_temp["totalCases"],labels=_temp["state"], shadow=True,radius=2,autopct='%1.1f%%',colors=colors)
        Axes.axis('equal')

        Figure.tight_layout()
        Figure.savefig('__temp/__fixed/pcbs.png')

        del date
        del _temp
        del _main
        del _others
        del colors
        Axes.cla()
        Figure.clear()
        plt.close()
        
    def MakeStateComparisonGraphs(self):
        plt.ioff()
        for i in self.states["State"]:
            for j in self.states["State"]:
                if i != j:
                    self.ComparisonInfectedBarState(i,j,(i+"_"+j))
                    self.ComparisonDeathsBarState(i,j,(i+"_"+j))

    
#     def MakeCitiesComparisonGraphs(self):
#         plt.ioff()
#         for i in self.cities.index:
#             for j in self.cities.loc[i]:
#                 if j != False:
#                     for k in self.cities.index:
#                         for l in self.cities.loc[k]:
#                             if l != False:
#                                 if j != l:
#                                     self.ComparisonInfectedBarCity(j,l,
#                                                                    (j+"_"+l))
#                                     self.ComparisonDeathsBarCity(j,l,
#                                                                  (j+"_"+l))
#                             else:
#                                 break
#                 else:
#                     break
    
    def MakeTemporalSeries(self):
        plt.ioff()
        for i in self.states["State"]:
            self.TemporalSeriesState(i,i)
            for j in self.cities.loc[i]:
                if j != False:
                    self.TemporalSeriesCity(j,j)
                else: 
                    break
                    

In [111]:
data = DataVisualizing(DataLoad())

In [112]:
data.PieCasesByState()